Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML: https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Beautiful Soup

**Beautiful Soup** es una librería **Python** que permite extraer información de contenido en formato **HTML o XML**. Para usarla, es necesario especificar un **parser**, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de **lxml**. Precisamente, será el que utilicemos en este tutorial.

**Será necesario instalar las siguientes librerías** (si no las tienes ya):

        pip3 install beautifulsoup4 requests pandas

        pip3 install beautifulsoup4 

        pip3 install requests

        pip3 install pandas 

###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

Algunos ejemplos de Web Scraping utilizando Beautiful Soup:

https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

http://omz-software.com/pythonista/docs/ios/beautifulsoup_guide.html

https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [ ]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [ ]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [ ]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [ ]:
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'lxml')

In [7]:
type(soup)

bs4.BeautifulSoup

In [ ]:
soup

In [8]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_ = 'product-image')
len(cervezas_grid)

12

In [10]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URL = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('a')['href']
    lista_URL.append(URL_cerveza)
lista_URL

['https://www.labirratorium.com/es/lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/alemania/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/inicio/199-orval.html',
 'https://www.labirratorium.com/es/alemania/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/165-laugar-epa.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html',
 'https://www.labirratorium.com/es/inicio/21-weihenstephaner-vitus.html']

In [11]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URL[0])
soup_cerveza = BeautifulSoup(r.text, 'lxml')

In [12]:
# Nombre
nombre = soup_cerveza.find(class_ = 'h1 product-detail-name').text
print(nombre)

Boon Oude Kriek 37,5cl


In [22]:
# Precio
precio = soup_cerveza.find(class_ = 'current-price').find('span')['content']
precio

'7.15'

In [16]:
# Descripcion corta
descrp_corta = soup_cerveza.find(class_ = 'description-short').find('p').text
descrp_corta

'Lambic / Kriek'

In [19]:
# Descripción larga
descrp_larga = soup_cerveza.find(class_ = 'product-description').find('p').text
descrp_larga

'Cerveza de fermentación espontánea (Lambic) de 6.5% ABV sin filtrar ni pasteurizar de estilo Kriek, elaborada con cerezas naturales.'

In [28]:
# Imagen
imagen = soup_cerveza.find(class_ = 'product-cover').find(id = 'zoom_product')['src']
imagen

'https://www.labirratorium.com/19351-large_default/boon-kriek-2013.jpg'

In [29]:
# Otra forma de obtener la imagen (así lo hizo Borja)

imagen2 = soup_cerveza.find(class_ ='js-qv-product-cover img-fluid')['src']
imagen2

'https://www.labirratorium.com/19351-large_default/boon-kriek-2013.jpg'

In [30]:
 # Brand

marca = soup_cerveza.find(class_ = 'img img-thumbnail manufacturer-logo')['alt']
marca

'Brouwerij F. Boon'

In [ ]:
# Código de barras



In [34]:
# Features

features = soup_cerveza.find(class_ = 'data-sheet')
features_dic = {}
features_dt = features.find_all('dt')
features_dd = features.find_all('dd')
for feature, value in zip(features_dt, features_dd):
    features_dic[feature.text] = value.text
features_dic

{'Estilo': 'KRIEK',
 'Origen': 'Bélgica',
 '% Alc.': '6.5\nALTO (6-9%)',
 'Otros ingredientes': 'Cerezas Naturales',
 'Volumen (cl)': '37.5 Cl',
 'Tipo Fermentación': 'Lambic (Fermentación espontánea o salvaje)',
 'Maltas': 'Cebada y Trigo',
 'IBU': '0-25 Amargor bajo',
 'Color': 'Rojiza',
 'Envase': 'Botella'}

# Probar otra manera de crear ese mismo diccionario de arriba

In [ ]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD

id_cerv = 'lbt_' + str()

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [ ]:
# Agregamos a una lista

pages = np.arrange(1,81)
count = 1
lista_cervezas = []

for page in pages:

    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    type(soup)

    # Guardamos lista de cervezas
    cervezas_grid = soup.find_all(class_ = 'product-image')
    len(cervezas_grid)

    # necesitamos acceder a cada una de las cervezas del grid:
    
    for cerveza in cervezas_grid:
        URL_cerveza = cerveza.find('a')['href']

    # Hacemos un nuevo request para la primera cerveza: 
        r = requests.get(lista_URL[0])
        soup_cerveza = BeautifulSoup(r.text, 'lxml')

    # Nombre
        nombre = soup_cerveza.find(class_ = 'h1 product-detail-name').text
        print(nombre)

        # Precio
        precio = soup_cerveza.find(class_ = 'current-price').find('span')['content']
        precio

        # Descripción corta
        try:
            descrp_corta = soup_cerveza.find(class_ = 'description-short').find('p').text
        except:
            descrp_corta = None

        # Descripción larga
        try:
            descrp_larga = soup_cerveza.find(class_ = 'product-description').find('p').text
        except:
            descrp_larga = None

        # Otra forma de obtener la imagen (así lo hizo Borja)
        imagen2 = soup_cerveza.find(class_ ='js-qv-product-cover img-fluid')['src']
        imagen2

        # Brand
        marca = soup_cerveza.find(class_ = 'img img-thumbnail manufacturer-logo')['alt']
        marca

        # Features
        features = soup_cerveza.find(class_ = 'data-sheet')
        features_dic = {}
        features_dt = features.find_all('dt')
        features_dd = features.find_all('dd')
        for feature, value in zip(features_dt, features_dd):
            features_dic[feature.text] = value.text
        features_dic



In [35]:
# CODIGO QUE PASO BORJA POR DISCORD

pages = np.arange(1,81)
count = 1
lista_cervezas = []

for page in pages:

    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_='product-image')

    count_beer = 1

    for cerveza in cervezas_grid:

        print('Cerveza {} de {}, pag {}/{}'.format(count_beer, len(cervezas_grid), page, len(pages)))

        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = BeautifulSoup(r.text, 'lxml')

        id_cerv = 'lbt_' + str(count)

        name = soup_cerveza.find(class_='h1 product-detail-name').text   

        price = soup_cerveza.find(class_='current-price').find('span')['content']

        try:
            descr_short = soup_cerveza.find(class_='description-short').find('p').text
        except:
            descr_short = None

        image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']

        try:
            brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            brand = None

        features_dicc = {}
        try:
            features = soup_cerveza.find(class_ = 'data-sheet')
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                #print(feature.text, value.text)
                features_dicc[feature.text] = value.text
        except:
            features_dicc = {}

        lista_cervezas.append([
            ('id', id_cerv),
            ('name', name), 
            ('price', price),
            ('descr_short', descr_short),
            ('image', image),
            ('brand', brand),
            ('features', features_dicc)])

        count += 1
        count_beer += 1

Cerveza 1 de 12, pag 1/80
Cerveza 2 de 12, pag 1/80
Cerveza 3 de 12, pag 1/80
Cerveza 4 de 12, pag 1/80
Cerveza 5 de 12, pag 1/80
Cerveza 6 de 12, pag 1/80
Cerveza 7 de 12, pag 1/80
Cerveza 8 de 12, pag 1/80
Cerveza 9 de 12, pag 1/80
Cerveza 10 de 12, pag 1/80
Cerveza 11 de 12, pag 1/80
Cerveza 12 de 12, pag 1/80
Cerveza 1 de 12, pag 2/80
Cerveza 2 de 12, pag 2/80
Cerveza 3 de 12, pag 2/80
Cerveza 4 de 12, pag 2/80
Cerveza 5 de 12, pag 2/80
Cerveza 6 de 12, pag 2/80
Cerveza 7 de 12, pag 2/80
Cerveza 8 de 12, pag 2/80
Cerveza 9 de 12, pag 2/80
Cerveza 10 de 12, pag 2/80
Cerveza 11 de 12, pag 2/80
Cerveza 12 de 12, pag 2/80
Cerveza 1 de 12, pag 3/80
Cerveza 2 de 12, pag 3/80
Cerveza 3 de 12, pag 3/80
Cerveza 4 de 12, pag 3/80
Cerveza 5 de 12, pag 3/80
Cerveza 6 de 12, pag 3/80
Cerveza 7 de 12, pag 3/80
Cerveza 8 de 12, pag 3/80
Cerveza 9 de 12, pag 3/80
Cerveza 10 de 12, pag 3/80
Cerveza 11 de 12, pag 3/80
Cerveza 12 de 12, pag 3/80
Cerveza 1 de 12, pag 4/80
Cerveza 2 de 12, pag 4/80
Cer

In [38]:
df = pd.DataFrame([[x[0][1],
                    x[1][1],
                    x[2][1],
                    x[3][1],
                    x[4][1],
                    x[5][1],
                    x[6][1]] for x in lista_cervezas],
                    columns = ['id', 'name', 'price', 'short drecript', 'image', 'brand', 'features'])

df.to_csv('df_labirratorium.csv', sep = ';')
df.head(30)

,id,name,price,short drecript,image,brand,features
0,lbt_1,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,https://www.labirratorium.com/19351-large_defa...,Brouwerij F. Boon,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
1,lbt_2,Störtebeker Schwarz-Bier,2.4,Cerveza negra de baja fermentación,https://www.labirratorium.com/488-large_defaul...,Störtebeker,"{'Estilo': 'SCHWARZBIER', 'Origen': 'Alemania'..."
2,lbt_3,Orval,2.8,Belgian Pale Ale,https://www.labirratorium.com/385-large_defaul...,Orval,"{'Estilo': 'BELGIAN PALE ALE', 'Origen': 'Bélg..."
3,lbt_4,Augustiner Lagerbier Hell,2.5,Lager / Helles,https://www.labirratorium.com/367-large_defaul...,Augustiner,"{'Estilo': 'MUNICH HELLES', 'Origen': 'Alemani..."
4,lbt_5,Schneider Aventinus Weizen-Eisbock,2.7,"Cerveza de color medio oscuro, aromas alcohóli...",https://www.labirratorium.com/366-large_defaul...,Schneider,"{'Origen': 'Alemania', '% Alc.': '12', 'Volume..."
5,lbt_6,Aecht Schlenkerla Rauchbier Weizen,2.5,,https://www.labirratorium.com/364-large_defaul...,Brauerei Heller,"{'Estilo': 'RAUCHBIER', 'Origen': 'Alemania', ..."
6,lbt_7,Samuel Adams Boston Lager,2.4,Cerveza de baja fermentación (Lager) de color ...,https://www.labirratorium.com/343-large_defaul...,Samuel Adams,"{'Estilo': 'AMERICAN LAGER', 'Origen': 'EEUU',..."
7,lbt_8,Laugar EPA! Euskadiko Pale Ale,2.65,American Pale Ale,https://www.labirratorium.com/11103-large_defa...,Laugar,"{'Estilo': 'APA (AMERICAN PALE ALE)', 'Origen'..."
8,lbt_9,Westmalle Dubbel,2.3,Dubbel,https://www.labirratorium.com/102-large_defaul...,Van Westmalle,"{'Estilo': 'DUBBEL', 'Origen': 'Bélgica', '% A..."
9,lbt_10,Duchesse De Bourgogne,2.95,Oud Bruin/ Flanders Red Ale,https://www.labirratorium.com/12300-large_defa...,Verhaeghe,{'Estilo': 'FLANDERS RED ALE OUD BRUIN Oud Bru...


### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

### FBI: Top ten criminals

#### Queremos guardar las imágenes de cada fugitivo y que el nombre de cada archivo sea el nombre del fugitivo:

In [ ]:
fbi_url = 'https://www.fbi.gov/wanted/topten'

